In [0]:
from pyspark.sql import functions as F

In [0]:
# Set paths
silver_table_path = "gs://databricks-glk-dbx-ext-storage/nyc_taxi_lakehouse/silver/silver_fhv_tripdata"
gold_table_path = "gs://databricks-glk-dbx-ext-storage/nyc_taxi_lakehouse/gold/gold_fhv_tripdata"

# --- Catalog/Schema (if Unity Catalog enabled) ---
catalog_name = "nyc_taxi_catalog"
schema_name  = "nyc_taxi_lakehouse"
table_name   = "gold_fhv_tripdata"

In [0]:
df_silver = spark.read.format("delta").load(silver_table_path)

df_gold = (
    df_silver
    .withColumn("month", F.date_format("pickup_datetime", "yyyy-MM"))
    .groupBy("month", "hvfhs_license_num")
    .agg(
        F.count("*").alias("total_trips"),
        F.avg("base_passenger_fare").alias("avg_base_fare"),
        F.avg("trip_miles").alias("avg_trip_miles"),
        F.avg("trip_time").alias("avg_trip_time_sec"),
        F.sum("base_passenger_fare").alias("total_fare"),
        F.sum("driver_pay").alias("total_driver_pay")
    )
    .withColumn("data_load_timestamp", F.current_timestamp())
)

df_gold.write.format("delta").partitionBy("month").option("overwriteSchema", "true").mode("overwrite").save(gold_table_path)

print(f"✅ Gold data written successfully to {gold_table_path}")

In [0]:
spark.sql(f"""
          CREATE TABLE IF NOT EXISTS {catalog_name}.{schema_name}.{table_name}
          USING DELTA
          LOCATION '{gold_table_path}'""")

In [0]:
spark.sql(f"select * from {catalog_name}.{schema_name}.{table_name}").display()

In [0]:
# Fully qualified table
catalog_name = "nyc_taxi_catalog"
schema_name = "nyc_taxi_lakehouse"
gold_table_name = "gold_fhv_tripdata"
fq_gold_table = f"{catalog_name}.{schema_name}.{gold_table_name}"

# Latest month just ingested (adjust per run)
latest_month = "2025-06"

# Optimize the latest partition
spark.sql(f"""
OPTIMIZE {fq_gold_table}
WHERE month = '{latest_month}'
ZORDER BY (hvfhs_license_num)
""")

# Compute stats
spark.sql(f"ANALYZE TABLE {fq_gold_table} COMPUTE STATISTICS")
spark.sql(f"ANALYZE TABLE {fq_gold_table} COMPUTE STATISTICS FOR COLUMNS month, hvfhs_license_num")


# Optional: vacuum old files
spark.sql(f"VACUUM {fq_gold_table} RETAIN 168 HOURS")

In [0]:
spark.sql(f"DESCRIBE DETAIL {fq_gold_table}").display()